<a href="https://colab.research.google.com/github/saahithhegde/NLP_PROJECT/blob/master/NLP_PART2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
cd 

/root


In [0]:
!pip install allennlp==1.0.0rc1 allennlp-models==1.0.0rc1

In [108]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

2020-05-04 21:10:54 WARNING: Did not use initialization regex that was passed: _context_layer._module.weight_hh.*
2020-05-04 21:10:54 WARNING: Did not use initialization regex that was passed: _context_layer._module.weight_ih.*


In [0]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza==1.0.0

# Import stanza
import stanza

In [0]:
!echo "Downloading CoreNLP..."
!wget "http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip" -O corenlp.zip
!unzip corenlp.zip
!mv ./stanford-corenlp-full-2018-10-05 ./corenlp

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = "./corenlp"

In [0]:
from stanza.server import CoreNLPClient

In [0]:
client = CoreNLPClient(annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner','coref','depparse'], memory='18G', endpoint='http://localhost:9001')
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

In [0]:
import spacy
from nltk.corpus import wordnet as wn
from spacy import displacy
from IPython.core.display import display, HTML
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

#Token Nlp has all the possible models | do not use it for sentence splitting
TokenNLP = spacy.load("en_core_web_sm")

#sentence nlp is used to explicitly spit the sentences 
SentenceNLP = spacy.load("en_core_web_sm", disable=["tagger", "ner"])
SentenceNLP.pipeline

#custom boundries
def custom_sentencizer(doc):
    for i, token in enumerate(doc[:-2]):
        # Define sentence start if pipe + titlecase token
        if token.text == "I":
            doc[i].is_sent_start = False
        if token.text == "(;":
            doc[i].is_sent_start = False
        if token.text == "(;":
            doc[i+1].is_sent_start = False
        if token.text == "...":
            doc[i+1].is_sent_start = True
        if token.text == ":":
            doc[i+1].is_sent_start = False
        if token.text == ":":
            doc[i].is_sent_start = False
        if token.text == "the":
            doc[i].is_sent_start = False
        if token.text == '."':
            doc[i+1].is_sent_start = True
        if token.text == '."':
            doc[i].is_sent_start = False
        if token.text == 'Inc.':
            doc[i].is_sent_start = False
        if token.text == 'Inc.':
            doc[i+1].is_sent_start = False
        if token.text == ', Inc.':
            doc[i+1].is_sent_start = False         
    return doc

SentenceNLP.add_pipe(custom_sentencizer, before="parser")
SentenceNLP.pipeline

In [0]:
text = "During his keynote speech at the Macworld Expo on January 9, 2007, Jobs announced that Apple Computer, Inc. would thereafter be known as 'Apple Inc.', because the company had shifted its emphasis from computers to consumer electronics. This event also saw the announcement of the iPhone and the Apple TV. The company sold 270,000 iPhone units during the first 30 hours of sales, and the device was called 'a game changer for the industry'. Apple would achieve widespread success with its iPhone, iPod Touch and iPad products, which introduced innovations in mobile phones, portable music players and personal computers respectively. Furthermore, by early 2007, 800,000 Final Cut Pro users were registered."
document = client.annotate(text)
print(document)

In [158]:
#to upload a file
from google.colab import files
uploaded=files.upload()
filename=None
for name, data in uploaded.items():
  filename=name

print(filename)

Saving 4.txt to 4.txt
4.txt


In [101]:
#when you have uploaded a file on the left
name=input("enter file name if you uploaded directly from the left")
filename=name+".txt"

enter file name if you uploaded directly from the leftsamplefilefordemo


In [0]:
#to upload a file 
with open(filename,encoding='utf-8-sig') as fd:
  lines = fd.read().splitlines()
  while '' in lines:
    lines.remove('')
print("The Document is split by \\n and appends each paragraph to the list:")
print(lines)

In [0]:
#lines are always the paragraphs:
lines=[]
myinput=input("Please enter some data")
lines.append(myinput)

In [0]:
pronounlist=["all","another","any","anybody","anyone","anything","as","aught","both","each","each other","either","enough","everybody","everyone","everything","few","he","her","hers","herself","him","himself","his","I","idem","it","its","itself","many","me","mine","most","my","myself","naught","neither","no one","nobody","none","nothing","nought","one","one another","other","others","ought","our","ours","ourself","ourselves","several","she","some","somebody","someone","something","somewhat","such","suchlike","that","thee","their","theirs","theirself","theirselves","them","themself","themselves","there","these","they","thine","this","those","thou","thy","thyself","us","we","what","whatever","whatnot","whatsoever","whence","where","whereby","wherefrom","wherein","whereinto","whereof","whereon","wherever","wheresoever","whereto","whereunto","wherewith","wherewithal","whether","which","whichever","whichsoever","who","whoever","whom","whomever","whomso","whomsoever","whose","whosever","whosesoever","whoso","whosoever","ye","yon","yonder","you","your","yours","yourself","yourselves"]

In [0]:
#finaljson append to extraction
final_json={
	"document": filename,
    "extraction": [
	]
}

In [0]:
#get only tags related to places only to categorize the sentences
def ner_for_places_only(sent):
  document = client.annotate(sent)
  ner_list=[]
  placeflag=False
  for sent in document.sentence:
    for m in sent.mentions:
      if m.entityType=="LOCATION" or m.entityType=="COUNTRY" or m.entityType=="STATE_OR_PROVINCE" or m.entityType=="CITY":
        placeflag=True
        ner_list+=[(m.entityMentionText, m.entityType , m.tokenStartInSentenceInclusive ,m.tokenEndInSentenceExclusive)]
        

  return ner_list,placeflag

In [0]:
def ner(sent):
  # ner_dict = {}
  doc = TokenNLP(sent)
  # print([(X.start, X.end) for X in doc.ents])
  ner_list = [(X.text, X.label_, X.start, X.end) for X in doc.ents]
  # ner_list = []
  # for X in doc.ents:
  #   if X.label_ == "MONEY":
  #     ner_list.append(X.text)
  # TODO custom tag for job titles

  return ner_list

In [0]:
def ner_usingstanford_and_job_flag(sent,doyouwantjobsonly=False):
  document = client.annotate(sent)
  ner_list=[]
  jobflag=False
  for sent in document.sentence:
    for m in sent.mentions:
        if doyouwantjobsonly is True:
          if(m.entityType=="TITLE"):
            jobflag=True
            ner_list+=[(m.entityMentionText, m.entityType , m.tokenStartInSentenceInclusive ,m.tokenEndInSentenceExclusive)]
        else:
          ner_list+=[(m.entityMentionText, m.entityType , m.tokenStartInSentenceInclusive ,m.tokenEndInSentenceExclusive)]

        if(m.entityType=="TITLE"):
          jobflag=True
        
  return ner_list,jobflag

In [0]:
# NER MONEY function
def get_money123(doc):  
  for X in doc.ents:
    if X.label_ == "MONEY":
      return X.text,True

  return "",False

In [0]:
# NER Quantity function
def get_quantity(doc):  
  for X in doc.ents:
    if X.label_ == "QUANTITY":
      return X.text

  return ""

In [0]:
# this function return the resolved text when it is passed to it:
def corefs(text):
  x=predictor.predict(document=text)
  #print(x)
  #occurance of the words
  tokens_of_sentence=x['document']
  #pronoun->noun dic
  noun_coref_instance={}
  #get the propernoun:
  for initalcluster in x["clusters"]:
    #print(initalcluster)
    noun_index=None
    for mynouns in initalcluster:
      y=mynouns
      noun_to_use_list=tokens_of_sentence[y[0]:y[1]+1]
      noun_to_use=' '.join([str(elem) for elem in noun_to_use_list]) 
      if noun_to_use.lower() in pronounlist:
        continue
      else:
       
        noun_index=y
        break

      
    #print(noun_index)
    #make a dictionary of prnouns and the resolved propernoun
    for pronouns_index in initalcluster:
      y=pronouns_index
      pronoun_to_use_list=tokens_of_sentence[y[0]:y[1]+1]
      pronoun_to_use=' '.join([str(elem) for elem in pronoun_to_use_list]) 
      if pronoun_to_use.lower() in pronounlist and noun_index is not None:
        noun_coref_instance[tuple(pronouns_index)]=noun_index

  #print(noun_coref_instance)

  for x in noun_coref_instance:
    #print(x)
    pronoun_value = tokens_of_sentence[x[0]:x[1]+1]
    # print(pronoun_value)
    repl_list_strt_idx = x[0]
    repl_list_end_idx = x[1]+1
    noun_to_use=noun_coref_instance.get(x)
    #print(y)
    new_list = tokens_of_sentence[noun_to_use[0]:noun_to_use[1]+1]
    listToStr = ' '.join([str(elem) for elem in new_list]) 
    # print(listToStr)
    tokens_of_sentence[repl_list_strt_idx : repl_list_end_idx] = [listToStr] 
    #print("----")

    #resolve the dictionary itselfokay wait 
    
  resolved=""
  for i in tokens_of_sentence:
    resolved+=str(i)+" "
  return resolved

In [0]:
#this part of the code resolves the lines/paragraph array with the coref:
for x,paragraph in enumerate(lines):
  if paragraph=="\n":
    continue
  print("-------------")
  print(paragraph)
  
  PARAPRO=False
  nlp = spacy.load("en_core_web_sm")
  docsentence = SentenceNLP(paragraph)
  for sent in docsentence.sents:
    
    doc = nlp(sent.text.strip())
    for token in doc: 
      # If pronoun exists in sentence, activate PRONFLAG
      if PARAPRO:
        break
      if token.pos_ == "PRON" or token.text in pronounlist:
          PARAPRO = True
          print("triggered pronoun in para")
          break
  if PARAPRO:
    resolved_paragraph=corefs(paragraph)
    lines[x]=resolved_paragraph

    finaldoc = SentenceNLP(lines[x])
    for sent in finaldoc.sents:
      print(sent.text.strip())
      print("-------------")

    gotdata=False
    try:
      nextparatocheck = lines[x+1]
      gotdata=True
    except IndexError:
      gotdata=False
      
    if gotdata:
      finaldoc_next = SentenceNLP(nextparatocheck)
      my_sentences_next=[]
      for sent in finaldoc_next.sents:
        my_sentences_next.append(sent.text.strip())


      # To check if Pronoun exists in sentence
    
      PRONFLAG = False
      WORDFLAG = False
    
      doc = nlp(my_sentences_next[0])
      for token in doc: 
        # If pronoun exists in sentence, activate PRONFLAG
        if token.pos_ == "PRON" or token.text in pronounlist:
            PRONFLAG = True
            print("triggered pronoun in next sentence")
            break
            
        if PRONFLAG:
          resolvedsentence = corefs(lines[x]+" mysplitter "+my_sentences_next[0])
          resolvedsentence=resolvedsentence.split(" mysplitter ")
          
          my_sentences_next[0]=resolvedsentence[1]
          print(resolvedsentence[1])

          nextpara=''
          for wordsin in my_sentences_next:
            nextpara.append(wordsin)
          lines[x+1]=nextpara


In [162]:
print(lines)

["As the market for personal computers expanded and evolved through the 1990s , Netflix lost market share to the lower - priced duopoly of Microsoft Windows on Intel PC clones . The company board hired CEO Mark Zuckerberg to what would be a 500-day charge for CEO Mark Zuckerberg to rehabilitate the financially troubled company — reshaping Netflix with layoffs , executive restructuring , and product focus . In 2001 , CEO Mark Zuckerberg led Netflix to acquire NeXT , solving the desperately failed operating system strategy and bringing Jim back . Jim slowing regained leadership status and became the CEO in 2002 . Netflix swiftly returned to profitability under the revitalizing Think different campaign , as Jim rebuilt Netflix 's status by launching the iMac in 1998 , opening the retail chain of Netflix Stores in 2001 , and acquiring numerous companies to broaden the software portfolio . In January 2007 , Jim renamed the company Netflix Inc. , reflecting Netflix shifted focus toward consu

In [0]:
#triggering mechanisms:
#going through each paragraph:
#lines=["In November 2009 , JPMorgan announced JPMorgan would acquire the balance of JPMorgan Cazenove , an advisory and underwriting joint venture established in 2004 with the Cazenove Group , for GBP1 billion ."]
checklist_for_buying=["buy", "get", "purchase", "acquire","acquisition","procurement","price","barter","obtain","secure","gain","garner"]
for paragraph in lines:
  docsentence = SentenceNLP(paragraph)
  for sent in docsentence.sents:
    sentence=sent.text.strip()
    print(sentence)
    doc = TokenNLP(sentence)
   
    #specify if you want only the job titles
    #all types of ner use the flags appropriatly 
    nerforjobsandflag=ner_usingstanford_and_job_flag(sentence,True)
    nerforplacesonly=ner_for_places_only(sentence)
    nerusingstanford=ner_usingstanford_and_job_flag(sentence,False)
    nerbasic=ner(sentence)
    money=get_money123(doc)
    quantity=get_quantity(doc)



    # print(nerforjobsandflag)
    # print(nerforplacesonly)
    # print(nerusingstanford)
    # print(nerbasic)
    # print(money)
    # print(quantity)

    BuyFlag=False
    PlaceFlag=False
    OrgFlag=False


    #trigger the buy template
    for w in checklist_for_buying:
      if w in [token.lemma_ for token in doc]:
        print("processing for buy")
        yo=buytepmlate(sentence)
        
        for i in yo:
          result = {}
          result["template"] = "BUY TEMPLATE"
          result["sentence"] = sentence
          result["arguments"] = i
          final_json["extraction"].append(result)
        break

   

    if nerforjobsandflag[1]==True:
      print("trigger job template triggered")
      myrs=worktemp(sentence)
      result = {}
      for i in myrs:
          result["template"] = "WORK TEMPLATE"
          result["sentence"] = sentence
          result["arguments"] = i
          final_json["extraction"].append(result)

    #trigger the place template 
    if(nerforplacesonly[1]==True):
      print("place template triggered")
      rs=place_template(sentence)
      for i in rs:
          result = {}
          result["template"] = "PLACE TEMPLATE"
          result["sentence"] = sentence
          result["arguments"] = i
          final_json["extraction"].append(result)


    #trigger the Work template




In [0]:
import spacy
from nltk.corpus import wordnet as wn
nlp = spacy.load("en_core_web_sm")

In [0]:
# NER MONEY function
def get_money(doc):  
  for X in doc.ents:
    if X.label_ == "MONEY":
      return X.text

  return ""

In [0]:
# BUY function
def buytepmlate(sentence):
  print("FUNC :: buytepmlate()")

  doc = nlp(sentence)

  buy_lemmea_list = ["buy", "get", "purchase", "acquire","acquisition","procurement","price","barter","obtain","secure","gain","garner"]
  for token in doc:
    if token.lemma_ in buy_lemmea_list:
      vflag = token.pos_ == "VERB"

  noun_chunks = list(doc.noun_chunks)

  sents = list(doc.sents)
  root = [s.root.text for s in sents][0]
  # print("ROOT:", root)
  # print("MONEY: ", get_money(doc))
  money = get_money(doc)
  quant=get_quantity(doc)
  for token in doc:
    buyer = ""
    seller_list= []
    quantity = ""
    if token.tag_ == "VBD" and token.dep_ == "ROOT" and vflag:
      # print("CASE 1")
      for i, chunk in enumerate(noun_chunks):    
        if chunk.root.dep_ == "nsubj" and chunk.root.head.text == root:
          # print("BUYER:", chunk.text)
          buyer = chunk.text 
        if chunk.root.dep_ == "dobj" and chunk.root.head.text == root:
          # print("SELLER:", chunk.text)
          seller_list.append(chunk.text)
        if chunk.root.dep_ == "appos" and chunk.root.head.text == noun_chunks[i-1].text.split(" ")[-1]:
          # print("SELLER:", chunk.text)
          seller_list.append(chunk.text)

      break

    # case 2: when lemma is VBN and ROOT
    if token.tag_ == "VBN" and token.dep_ == "ROOT" and vflag:
      for chunk in noun_chunks:
        if chunk.root.dep_ == "nsubjpass" and chunk.root.head.text == root:
          # print("SELLER:", chunk.text)
          seller_list.append(chunk.text)
        if chunk.root.dep_ == "pobj" and chunk.root.head.text == "by":
          # print("BUYER:", chunk.text)
          buyer = "chunk.text"  
      

      break
    
    # case 3: when lemma is NN, find noun chunks
    if token.tag_ == "NN" and token.dep_ == "dobj" and not vflag:
      for chunk in noun_chunks:

        if chunk.root.dep_ == "nsubj" and chunk.root.head.text == root:
          # print("BUYER:", chunk.text)
          buyer = chunk.text
        if chunk.root.dep_ == "pobj" and chunk.root.head.text == "of":
          # print("SELLER:", chunk.text)
          seller_list.append(chunk.text)
      break

  rs = []
  for s in seller_list:
    res = {}
    res["buyer"] = buyer
    res["source"] = s
    res["money"] = money
    res["quantity"] = quant
    res["item"]=""

    rs.append(res)
      
  return rs


In [0]:
# PLACE template

def place_template(sentence):

  subloc = []
  suploc = []

  doc = nlp(sentence)
  # for token in doc:
  #   print(token.text, "-->", token.lemma_, token.pos_, token.tag_, token.dep_, "head: ", token.head, token.head.dep_)
  #   print(token.text, token.conjuncts)

  noun_chunks = list(doc.noun_chunks)
  # print("\n---NOUN_CHUNKS---\n", noun_chunks)

  # for token in doc:
  #   if token.pos_ == "PROPN" and token. not in noun_chunks:
  #     noun_chunks.append(token)

  # print("chunk.text", ",", "chunk.root.text", ",", "chunk.root.dep_", ",", "chunk.root.head.text", ", ", "chunk.root.pos_", "chunk.root.tag_", "chunk.root.head.dep_", "\nroot's root:", "chunk.root.head.head")
  # for chunk in noun_chunks:
  #     print(chunk.text, ",", chunk.root.text, ",", chunk.root.dep_, ",", chunk.root.head.text, ", ", chunk.root.pos_, chunk.root.tag_, chunk.root.head.dep_, "\nroot's root:", chunk.root.head.head)


  for i, chunk in enumerate(noun_chunks):
    if chunk.root.pos_ == "PROPN":
      # if chunk.root.head.text == "is" or chunk.root.head.text == "are":
      if search_is(chunk.root) and not search_in(chunk.root) and not search_of(chunk.root):
        # This part makes "City 'of' New York work"
        if i+1 < len(noun_chunks) and noun_chunks[i+1].root.head.text == "of" and noun_chunks[i+1].root.dep_ == "pobj": 
          # print("1SUBLOC:", chunk.text + " of " + noun_chunks[i+1].text, "i: ", i)
          subloc.append(chunk.text + " of " + noun_chunks[i+1].text)
        else:
          # print("2SUBLOC:", chunk.text, "i: ", i)
          subloc.append(chunk.text)


    if chunk.root.pos_ == "PROPN" or chunk.root.pos_ == "NOUN":
      if chunk.root.head.text == "in":
        
        if  chunk.root.dep_ == "pobj":        
          if i+1 < len(noun_chunks) and noun_chunks[i+1].root.head.text == "of" and noun_chunks[i+1].root.dep_ == "pobj": # and noun_chunks[i+1].root.pos_ == "PROPN":
            # print("4SUPERLOC:", chunk.text + " of " + noun_chunks[i+1].text, "i: ", i)
            suploc.append(chunk.text + " of " + noun_chunks[i+1].text)
          else:
            # print("3SUPERLOC:", chunk.text, "i: ", i)
            suploc.append(chunk.text)


      if chunk.root.tag_ == "NNP" and chunk.root.dep_ == "conj" and chunk.root.head.head.text == "in":
          # print("7SUPERLOC:", chunk)
          suploc.append(chunk.text)
    if chunk.root.head.text == "is" and chunk.root.head.dep_ == "relcl":
      # print("6SUBLOC::", noun_chunks[i-1], "i: ", i)
      subloc.append(noun_chunks[i-1])

    if chunk.root.head.text == "of" and chunk.root.head.head.text == "part":
      # print("5SUPERLOC:", chunk.text)
      suploc.append(chunk.text)

    if chunk.root.head.text == "of" and chunk.root.head.head.text == "state":
      # print("8SUBLOC:", chunk.text)
      subloc.append(chunk.text)
      # print("8SUPERLOC:", noun_chunks[i-1].text[0:-6])
      suploc.append(noun_chunks[i-1].text[0:-6])

    if chunk.root.pos_ == "PROPN" and chunk.root.head.pos_ == "PROPN":
      # print("SUBLOC:", chunk.root.text)
      subloc.append(chunk.root.text)
      # print("SUPERLOC:", chunk.root.head.text)
      suploc.append(chunk.root.head.text)

  result = []
  for sp in suploc:
    for sb in subloc:
      rs = {}
      rs["super-location"] = sp
      rs["sub-location"] = sb
      result.append(rs)
  return result

In [0]:
def worktemp(sentence):
  doc = nlp(sentence)

  reslist = []
  # words = ["of", "at", "as"]
  #print(doc)
  for token in doc:
    if token.text == "of" :
      res = {}
      # print("\n",token.text, "::","\nancestors:", [anc for anc in token.ancestors], 
      #       "\nchildren:",[(child, child.pos_) for child in token.children], "\nsubtree:", [s for s in token.subtree],
      #       "\nlefts:", [token.text for token in doc[2].lefts])
      
      #print("PERSON:", " ".join([token.text for token in doc[2].lefts]))
      res["person"] = " ".join([token.text for token in doc[2].lefts])
      
      joblist = []
      for anc in token.ancestors:
        if anc.pos_ in ["NOUN", "PROPN", "ADJ"] and anc.text not in joblist:
          joblist.append(anc.text)
      #print("POSTITION:", ", ".join(joblist))
      res["titles"] = str(joblist)
      # print("ORG:"," ".join([s.text for s in token.subtree][1:]))

      sc_flag = False
      for i, s in enumerate(token.subtree):
        if s.text == ";":
          pst_sc = i
          sc_flag = True
          break
        
      if sc_flag:
        #print("ORG:"," ".join([s.text for s in token.subtree][1:pst_sc+1]))
        res["ORG"] = " ".join([s.text for s in token.subtree][1:pst_sc+1])
      else:
        #print("ORG:"," ".join([s.text for s in token.subtree][1:]))
        res["ORG"] = " ".join([s.text for s in token.subtree][1:])
      #print("----")
      reslist.append(res)
      # print("tag:", token.tag_, "pos:", token.pos_)

    if token.text == "as":
      res = {}
      # print("\n",token.text, "::","\nancestors:", [anc for anc in token.ancestors], 
      #       "\nchildren:",[(child, child.pos_) for child in token.children], "\nsubtree:", [s for s in token.subtree],
      #       "\nlefts:", [token.text for token in doc[2].lefts])
      
      #print("PERSON:", " ".join([token.text for token in doc[2].lefts]))
      res["person"] = " ".join([token.text for token in doc[2].lefts])

      joblist = []
      for anc in token.ancestors:
        if anc.pos_ in ["NOUN", "PROPN", "ADJ"] and anc.text not in joblist:
          joblist.append(anc.text)
      #print("POSTITION:", ", ".join(joblist))
      res["titles"] = str(joblist)

      of_flag = False
      for i, s in enumerate(token.subtree):
        if s.text == "of":
          pst_of = i
          of_flag = True
        
      if of_flag:
        #print("ORG:"," ".join([s.text for s in token.subtree][pst_of+1:]))
        res["ORG"] = " ".join([s.text for s in token.subtree][pst_of+1:])
      else:
        #print("ORG:"," ".join([s.text for s in token.subtree][1:]))
        res["ORG"] = " ".join([s.text for s in token.subtree][1:])
      #print("----")
      reslist.append(rs)
  #print(reslist)
  return reslist

In [0]:
def search_is(val):
  while True:
    # print(val)
    if val.head.text == "is" or val.head.text == "are":
      return True
    elif val.head.text == val.head.head.text:
      return False 
    val = val.head

def search_in(val):    
  while True:
    if val.head.text == "in" or val.head.text == "In":
      return True
    elif val.head.text == val.head.head.text:
      return False 
    val = val.head

def search_of(val):    
  while True:
    if val.head.text == "of":
      return True
    elif val.head.text == val.head.head.text:
      return False 
    val = val.head

def search_including(val):    
  while True:
    if val.head.text == "including" or val.head.text == "including":
      return True
    elif val.head.text == val.head.head.text:
      return False 
    val = val.head

In [0]:
import io, json
print(final_json)
json_object = json.dumps(final_json, indent = 3) 

In [0]:

with io.open(filename+'_output.json', 'w') as f:
  f.write(json_object)
